In [2]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  # accessor needed to parse crs
import calendar
import argparse
import seaborn as sns
from shapely.geometry import Point
import geopandas
from geodatasets import get_path
import glob

In [31]:

# vegc_paths = glob.glob(r'/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_'+var+'{12530}/*')
# soilc_paths = glob.glob(r'/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_'+var+'12530*')
# forest_paths = vegc_paths + soilc_paths
# forest_paths
# var = '[GPP|NPP]'
# number = [12530, 13783, 15036, 16289, 17542]
# var = '[SOILORGC|VEGC]'
# soilc_paths = glob.glob(r'/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_'+var+'*')
# soilc_paths
files = [
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC12530.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC12530.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC13783.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC13783.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC15036.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC15036.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC16289.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC16289.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC17542.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC17542.csv'   

]
files
# print(len(vegc_paths), ' ', len(soilc_paths))

['/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC12530.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC12530.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC13783.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC13783.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC15036.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC15036.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC16289.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC16289.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC17542.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC17542.csv']

In [33]:
all_grids = pd.DataFrame( columns = [ 'year', 'lon', 'lat', 'variable','current_veg',
       'cohort_area', 'land_area', 'value',  'temp', 'value_weight'])

# a
for file in files:
    grids = pd.read_csv(file)
    
    print(grids.value.mean())
    print(grids.value.max())

    if len(grids) >0:
        aggregations = {
         'cohort_area':'sum',
         'land_area':'sum',
         'value':'mean',
            'temp':'sum'
        }
        grids['temp'] = grids['value_weight'] * grids['cohort_area'] 


        grids = grids.groupby([
               'year', 'lon', 'lat', 'variable','current_veg']).agg(
        aggregations
        )


        # all_grids =  all_grids.reset_index()

        grids['value_weight'] = grids['temp']/grids['cohort_area']
        grids = grids.reset_index()

        all_grids = pd.concat([all_grids, grids])
        
        
    print(file)
    print(len(all_grids))
all_grids.to_csv('amazon_loop_output_vegc_soilc_historical.csv')    

11326.046899097038
37429.41
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC12530.csv
352090
5766.03361230075
99053.99
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC12530.csv
704180
11532.777738814313
60236.68
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC13783.csv
1005384
9695.934862885195
363131.8358333334
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC13783.csv
1306588
7999.481325613213
31485.755
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC15036.csv
1782552
3885.1199655640257
33347.96
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC15036.csv
2258516
6667.4740061878665
32440.0
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC16289.csv
2658922
3542.0595119547884
22261.41
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts